In [ ]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
from srv_interface.srv import Order, Confirm
from rclpy.qos import QoSProfile

import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QListWidget, QStackedWidget, QPushButton,
    QVBoxLayout, QLabel, QDialog, QHBoxLayout, QWidget, QGridLayout, QScrollArea,
    QListWidgetItem, QMessageBox
)
from PyQt5.QtGui import QFont, QPixmap
from PyQt5.QtCore import Qt


class TableMonitorNode(Node):
    """
    ROS2 노드: 테이블 모니터 역할을 수행합니다.
    - 고객의 요청 사항을 퍼블리시합니다.
    - 주방에 주문을 서비스 요청합니다.
    - 배달 확인 요청에 대한 서비스를 제공합니다.
    """
    def __init__(self):
        super().__init__('table_monitor')  # 노드 이름 설정
        qos_profile = QoSProfile(depth=10)  # QoS 설정

        # Publisher: 고객 요청사항을 'table_request' 토픽에 발행
        self.publisher = self.create_publisher(String, 'table_request', qos_profile)

        # Client: 'kitchen_order' 서비스에 주문 요청
        self.request_client = self.create_client(Order, 'kitchen_order')

        # Server: 배달 확인 요청에 응답
        self.response_service = self.create_service(Confirm, 'delivery_confirm', self.handle_response)

        self.get_logger().info("Table Monitor가 실행 중입니다")

    def publish_request(self, request_type):
        """
        고객 요청사항을 'table_request' 토픽에 발행합니다.
        :param request_type: 고객이 요청한 서비스 (예: '물', '앞치마' 등)
        """
        msg = String()
        msg.data = request_type  # 요청 내용을 메시지에 담기
        self.publisher.publish(msg)  # 메시지 발행
        self.get_logger().info(f"고객 요청사항 발행: {request_type}")

    def send_order_request(self, table_num, menu, quantity):
        """
        주방에 주문 요청을 전송합니다.
        :param table_num: 테이블 번호
        :param menu: 주문한 메뉴 이름
        :param quantity: 주문 수량
        """
        # 주문 서비스가 사용 가능한지 확인
        if not self.request_client.wait_for_service(timeout_sec=3.0):
            self.get_logger().error("주문 서비스가 현재 사용 불가능합니다!")
            return

        # 주문 요청 생성
        request = Order.Request()
        request.table_num = table_num
        request.menu = menu
        request.quantity = quantity

        # 주문 서비스 호출
        future = self.request_client.call_async(request)
        rclpy.spin_until_future_complete(self, future)  # 응답 대기

        # 응답 처리
        if future.result():
            response = future.result()
            if response.accept:
                self.get_logger().info(f"주문 접수 완료: {menu} x {quantity}")
            else:
                self.get_logger().warning("주문이 거절되었습니다!")
        else:
            self.get_logger().error("서비스 응답 실패!")

    def handle_response(self, request, response):
        """
        배달 확인 요청에 응답합니다.
        :param request: 배달 확인 요청 (메뉴 정보 포함)
        :param response: 배달 확인 응답 (수락 여부)
        """
        self.get_logger().info(f"배달 확인 요청: {request.menu}")
        response.accept = True  # 배달을 수락으로 설정
        self.get_logger().info("배달 확인 응답 전송 완료")
        return response  # 응답 반환


class MainWindow(QMainWindow):
    """
    PyQt5 기반의 GUI 클래스: 테이블 모니터 역할을 수행합니다.
    - 메뉴 선택 및 주문 기능을 제공합니다.
    - 요청 사항을 전송할 수 있습니다.
    - 장바구니를 관리하고 결제 기능을 제공합니다.
    """
    def __init__(self, ros_node):
        super().__init__()
        self.cart = []  # 장바구니 데이터 저장
        self.table_num = 1  # 테이블 번호 (고정값으로 설정되어 있으나 필요에 따라 변경 가능)
        self.ros_node = ros_node  # ROS2 노드 연결
        
        # GUI 설정
        self.setWindowTitle("Restaurant Serving Robot")  # 윈도우 제목 설정
        self.setGeometry(100, 100, 2560, 1280)  # 윈도우 크기 및 위치 설정

        # 중앙 위젯과 레이아웃 설정
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        main_layout = QHBoxLayout(self.central_widget)  # 전체 수평 레이아웃 설정

        # 왼쪽 영역: 메뉴 카테고리 및 요청사항 버튼
        left_layout = QVBoxLayout()
        self.menu_list = QListWidget()  # 메뉴 카테고리 리스트 위젯
        self.menu_list.addItems(["메인", "음료", "주류"])  # 카테고리 추가
        self.menu_list.itemClicked.connect(self.change_menu)  # 카테고리 선택 시 이벤트 연결
        left_layout.addWidget(self.menu_list)

        self.request_button = QPushButton("요청사항")  # 요청사항 버튼
        self.request_button.clicked.connect(self.open_request_dialog)  # 클릭 시 이벤트 연결
        left_layout.addWidget(self.request_button)

        left_container = QWidget()
        left_container.setLayout(left_layout)
        main_layout.addWidget(left_container, stretch=1)  # 왼쪽 영역을 메인 레이아웃에 추가

        # 중앙 영역: 메뉴 선택 영역
        self.menu_stacked_widget = QStackedWidget()  # 스택 위젯을 사용하여 메뉴 페이지 전환
        self.setup_menu_pages()  # 메뉴 페이지 설정
        main_layout.addWidget(self.menu_stacked_widget, stretch=6)  # 중앙 영역을 메인 레이아웃에 추가

        # 오른쪽 영역: 장바구니 및 결제 버튼
        right_layout = QVBoxLayout()
        self.cart_widget = QListWidget()  # 장바구니 리스트 위젯
        self.cart_widget.itemClicked.connect(self.confirm_cancel)  # 장바구니 항목 클릭 시 취소 확인
        right_layout.addWidget(self.cart_widget)

        self.total_price_label = QLabel("총 가격: 0원")  # 총 가격 레이블 추가
        self.total_price_label.setFont(QFont("Arial", 12))  # 폰트 설정
        right_layout.addWidget(self.total_price_label)

        self.checkout_button = QPushButton("접수")  # 접수 버튼
        self.checkout_button.clicked.connect(self.checkout)  # 클릭 시 이벤트 연결
        right_layout.addWidget(self.checkout_button)

        right_container = QWidget()
        right_container.setLayout(right_layout)
        main_layout.addWidget(right_container, stretch=2)  # 오른쪽 영역을 메인 레이아웃에 추가

    def setup_menu_pages(self):
        """
        각 메뉴 카테고리에 대한 페이지를 설정합니다.
        """
        # 메뉴 데이터 정의 (이미지 파일 경로 포함)
        menu = {
            "메인": [
                {"name": "치킨","price": 18000, "image": "./src/images/chicken.jpg"},
                {"name": "피자", "price": 28000, "image": "./src/images/pizza.jpeg"},
                {"name": "햄버거", "price": 9000, "image": "./src/images/burger.jpg"},
                {"name": "스테이크", "price": 20000, "image": "./src/images/steak.jpg"},
                {"name": "파스타", "price": 12000, "image": "./src/images/pasta.jpeg"},
                {"name": "라자냐", "price": 11000, "image": "./src/images/lasagna.jpg"},
                {"name": "비프 웰링턴","price": 65000, "image": "./src/images/beef_wellington.jpg"},
            ],
            "음료": [
                {"name": "제로 콜라", "price": 2000, "image": "./src/images/cola.jpg"},
                {"name": "스프라이트", "price": 2000, "image": "./src/images/sprite.png"},
                {"name": "환타", "price": 2000, "image": "./src/images/fanta.png"},
            ],
            "주류": [
                {"name": "하이볼", "price": 5000, "image": "./src/images/highball.jpeg"},
                {"name": "생맥주 500cc", "price": 5000, "image": "./src/images/beer.jpg"},
                {"name": "소주", "price": 6000, "image": "./src/images/soju.png"},
            ],
        }

        self.pages = {}  # 카테고리별 페이지를 저장할 딕셔너리

        for category, items in menu.items():
            # 스크롤 영역 생성
            scroll_area = QScrollArea()
            scroll_area.setWidgetResizable(True)  # 내용 크기에 따라 스크롤 조정
            
            page = QWidget()
            page_layout = QGridLayout(page)  # 2열 그리드 레이아웃
            page_layout.setAlignment(Qt.AlignTop)  # 위쪽 정렬

            for i, item in enumerate(items):
                # 이미지 생성
                image_label = QLabel()
                pixmap = QPixmap(item["image"])  # 이미지 로드
                pixmap = pixmap.scaled(500, 400, Qt.IgnoreAspectRatio)  # 크기 조정
                image_label.setPixmap(pixmap)

                # 버튼 생성
                menu_button = QPushButton(f'{item["name"]}\n가격: {item["price"]}')
                menu_button.setFixedSize(500, 100)  # 버튼 크기 설정
                menu_button.clicked.connect(lambda _, name=item["name"], price=item["price"]: self.open_quantity_dialog(name, price))

                # 이미지와 버튼을 수직으로 정렬
                container = QVBoxLayout()
                container.addWidget(image_label)
                container.addWidget(menu_button)

                # 컨테이너를 위젯으로 감싸고 그리드 레이아웃에 추가
                widget = QWidget()
                widget.setLayout(container)
                page_layout.addWidget(widget, i // 2, i % 2, alignment=Qt.AlignCenter)  # 2열로 배치

            # 스크롤 영역에 페이지 설정
            page.setLayout(page_layout)
            scroll_area.setWidget(page)

            # 페이지를 스택 위젯에 추가
            self.menu_stacked_widget.addWidget(scroll_area)
            self.pages[category] = scroll_area

    def change_menu(self, item):
        """
        메뉴 카테고리를 변경합니다.
        :param item: 선택된 QListWidgetItem (카테고리 이름)
        """
        category = item.text()
        if category in self.pages:
            self.menu_stacked_widget.setCurrentWidget(self.pages[category])  # 선택된 카테고리의 페이지로 전환

    def open_quantity_dialog(self, menu_name, price):
        """
        메뉴 수량을 결정하는 다이얼로그를 엽니다.
        :param menu_name: 선택된 메뉴의 이름
        """
        dialog = QDialog(self)
        dialog.setWindowTitle(f"{menu_name} 수량 결정")
        
        # 팝업창 크기 설정
        dialog.setFixedSize(400, 300)  # 팝업창 크기를 가로 400px, 세로 300px로 설정
        layout = QVBoxLayout(dialog)

        quantity_label = QLabel("수량: 1")  # 초기 수량은 1로 설정
        layout.addWidget(quantity_label)
        count = 1  # 수량을 저장할 변수

        # 수량 증가 함수
        def increase_quantity():
            nonlocal count
            count += 1
            quantity_label.setText(f"수량: {count}")

        # 수량 감소 함수
        def decrease_quantity():
            nonlocal count
            if count > 1:
                count -= 1
                quantity_label.setText(f"수량: {count}")

        # 수량 조절 버튼 생성
        plus_button = QPushButton("+")
        plus_button.clicked.connect(increase_quantity)
        minus_button = QPushButton("-")
        minus_button.clicked.connect(decrease_quantity)
        layout.addWidget(plus_button)
        layout.addWidget(minus_button)

        # 확인 버튼: 장바구니에 메뉴 추가
        confirm_button = QPushButton("확인")
        confirm_button.clicked.connect(lambda: [self.add_to_cart(menu_name, count, count * price), dialog.accept()])
        layout.addWidget(confirm_button)

        dialog.setLayout(layout)
        dialog.exec_()  # 다이얼로그 실행

    def add_to_cart(self, menu_name, count, money):
        """
        선택된 메뉴와 수량을 장바구니에 추가합니다.
        :param menu_name: 메뉴 이름
        :param count: 선택된 수량
        """
        for item in self.cart:
            if item['menu'] == menu_name:
                item['quantity'] += count  # 수량 업데이트
                item['all_price'] += money
                break
        else:
            self.cart.append({"menu": menu_name, "quantity": count, "all_price": money})
        # 장바구니 UI 업데이트
        self.update_cart_view()

    def confirm_cancel(self, item):
        """
        장바구니 항목 클릭 시 취소 여부를 묻는 다이얼로그를 띄웁니다.
        :param item: 클릭된 장바구니 항목
        """
        # 취소 여부를 묻는 메시지 박스 생성
        reply = QMessageBox.question(self, '취소 확인', f"{item.text()} 메뉴를 장바구니에서 삭제하시겠습니까?",
                                     QMessageBox.Yes | QMessageBox.No, QMessageBox.No)

        if reply == QMessageBox.Yes:
            self.remove_from_cart(item)  # 장바구니에서 항목 제거

    def remove_from_cart(self, item):
        """
        장바구니에서 메뉴 항목을 제거합니다.
        :param item: 제거할 장바구니 항목
        """
        cart_item = item.data(Qt.UserRole)  # 장바구니 아이템 데이터 가져오기
        self.cart.remove(cart_item)  # 장바구니에서 항목 제거
        self.update_cart_view()  # 장바구니 UI 업데이트

    def update_cart_view(self):
        """
        장바구니 리스트 위젯을 업데이트합니다.
        """
        self.cart_widget.clear()  # 기존 항목 제거
        total_price = 0  # 총 가격 변수 초기화
        for item in self.cart:
            cart_item_widget = QListWidgetItem(f"{item['menu']} - 수량: {item['quantity']} - 총 가격: {item['all_price']}")  # 장바구니 아이템 표시
            cart_item_widget.setData(Qt.UserRole, item)  # 아이템 데이터를 저장 (수정할 때 사용)
            self.cart_widget.addItem(cart_item_widget)
            total_price += item['all_price']  # 총 가격 업데이트

        self.total_price_label.setText(f"총 가격: {total_price}원")  # 총 가격 레이블 업데이트

    def checkout(self):
        """
        결제 버튼 클릭 시 호출됩니다.
        장바구니에 있는 모든 메뉴를 주문 요청으로 전송합니다.
        """
        for item in self.cart:
            self.ros_node.send_order_request(self.table_num, item['menu'], item['quantity'])  # 주문 요청 전송
        self.cart.clear()  # 장바구니 비우기
        self.update_cart_view()  # 장바구니 뷰 업데이트

    def open_request_dialog(self):
        """
        고객 요청사항을 선택할 수 있는 다이얼로그를 엽니다.
        """
        dialog = QDialog(self)
        dialog.setWindowTitle("요청사항")
        layout = QVBoxLayout(dialog)

        # 요청사항 목록
        requests = ["물", "앞치마", "앞접시", "물티슈", "숟가락 & 젓가락", "직원 호출"]
        active_requests = set()  # 선택된 요청사항을 저장할 집합

        # 요청사항 버튼 생성 및 클릭 이벤트 연결
        def toggle_request(button, request):
            if request in active_requests:
                active_requests.remove(request)
                button.setStyleSheet("background-color: none;")  # 선택 해제 시 배경색 제거
            else:
                active_requests.add(request)
                button.setStyleSheet("background-color: lightblue;")  # 선택 시 배경색 변경

        for request in requests:
            button = QPushButton(request)
            button.setCheckable(True)
            button.clicked.connect(lambda _, b=button, r=request: toggle_request(b, r))
            layout.addWidget(button)

        # 확인 버튼: 선택된 요청사항을 퍼블리시
        confirm_button = QPushButton("확인")
        confirm_button.clicked.connect(lambda: self.confirm_requests(dialog, active_requests))
        layout.addWidget(confirm_button)

        dialog.setLayout(layout)
        dialog.exec_()  # 다이얼로그 실행

    def confirm_requests(self, dialog, active_requests):
        """
        선택된 요청사항을 퍼블리시합니다.
        :param dialog: 현재 다이얼로그 객체
        :param active_requests: 선택된 요청사항 집합
        """
        for request in active_requests:
            message = f"{self.table_num}번 Table 요청: {request}"
            self.ros_node.publish_request(message)  # 요청사항 퍼블리시
        dialog.accept()  # 다이얼로그 닫기


def main(args=None):
    """
    프로그램의 진입점 함수입니다.
    ROS2 노드를 초기화하고, PyQt5 애플리케이션을 실행합니다.
    """
    rclpy.init(args=args)  # ROS2 초기화
    ros_node = TableMonitorNode()  # ROS2 노드 생성

    app = QApplication(sys.argv)  # PyQt5 애플리케이션 생성
    main_window = MainWindow(ros_node)  # 메인 윈도우 생성
    main_window.show()  # 윈도우 표시

    app.exec_()  # 이벤트 루프 실행
    ros_node.destroy_node()  # ROS2 노드 종료
    rclpy.shutdown()  # ROS2 종료


if __name__ == "__main__":
    main()  # 프로그램 실행


In [ ]:
import sys
import threading
import queue
import rclpy
from rclpy.node import Node
from rclpy.qos import QoSProfile
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QPushButton, QVBoxLayout, QHBoxLayout,
    QGridLayout, QLabel, QListWidget, QWidget, QTextEdit, QDialog, QMessageBox, 
    QLineEdit, QTableWidget, QTableWidgetItem
)
from PyQt5.QtCore import Qt, QTimer
from std_msgs.msg import String
from srv_interface.srv import Order, Confirm
import os
import sqlite3
from datetime import datetime

from nav2_msgs.action import NavigateToPose
from rclpy.action import ActionClient

menu = {
        "치킨": {"price": 18000, "image": "./src/images/chicken.jpg"},
        "피자": {"price": 28000, "image": "./src/images/pizza.jpeg"},
        "햄버거": {"price": 9000, "image": "./src/images/burger.jpg"},
        "스테이크": {"price": 20000, "image": "./src/images/steak.jpg"},
        "파스타": {"price": 12000, "image": "./src/images/pasta.jpeg"},
        "라자냐": {"price": 11000, "image": "./src/images/lasagna.jpg"},
        "비프 웰링턴": {"price": 65000, "image": "./src/images/beef_wellington.jpg"},
        "제로 콜라": {"price": 2000, "image": "./src/images/cola.jpg"},
        "스프라이트": {"price": 2000, "image": "./src/images/sprite.png"},
        "환타": {"price": 2000, "image": "./src/images/fanta.png"},
        "하이볼": {"price": 5000, "image": "./src/images/highball.jpeg"},
        "생맥주 500cc": {"price": 5000, "image": "./src/images/beer.jpg"},
        "소주": {"price": 6000, "image": "./src/images/soju.png"},
        }

class KitchenNode(Node):
    """
    ROS2 Node: 주방 기능 처리
    """
    def __init__(self, request_queue):
        super().__init__('kitchen_node')  # 노드 이름 설정
        self.request_queue = request_queue  # 메시지를 전달받을 큐 생성
        qos_profile = QoSProfile(depth=10)  # QoS 설정

        # 테이블 요청 구독: 'table_request' 토픽에서 메시지를 받아 처리
        self.table_request_sub = self.create_subscription(
            String, 'table_request', self.handle_table_request, qos_profile)

        # 주문 서비스 서버: 'kitchen_order' 서비스 요청을 받아 처리
        self.order_service = self.create_service(Order, 'kitchen_order', self.handle_order)

        # 배달 확인 서비스 클라이언트: 'delivery_confirm' 서비스 요청 전송
        self.confirm_client = self.create_client(Confirm, 'delivery_confirm')

        self._action_client = ActionClient(self, NavigateToPose, 'navigate_to_pose')
        self.moving_path = []  # 경로를 저장하는 리스트
        self.busy = False  # 로봇 상태 플래그

        self.get_logger().info("KitchenNode가 초기화되었습니다.")

    def send_goal(self, x, y, theta):
        goal_msg = NavigateToPose.Goal()
        goal_msg.pose.header.frame_id = "map"
        goal_msg.pose.header.stamp = self.get_clock().now().to_msg()
        goal_msg.pose.pose.position.x = x
        goal_msg.pose.pose.position.y = y
        goal_msg.pose.pose.orientation.z = theta
        goal_msg.pose.pose.orientation.w = 1.0
        self._action_client.wait_for_server()
        self.get_logger().info(f'Sending goal to: x={x}, y={y}, theta={theta}')
        self._action_client.send_goal_async(
            goal_msg,
            feedback_callback=self.feedback_callback
        ).add_done_callback(self.goal_response_callback)

    def feedback_callback(self, feedback_msg):
        feedback = feedback_msg.feedback
        self.get_logger().info(f"Current progress: {feedback.current_pose}")

    def goal_response_callback(self, future):
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().info('Goal rejected :(')
            return
        self.get_logger().info('Goal accepted :)')
        goal_handle.get_result_async().add_done_callback(self.get_result_callback)

    def get_result_callback(self, future):
        result = future.result().result
        self.get_logger().info(f'Result: {result}')
        self.busy = False  # 목표 완료, 상태 해제
        self.send_next_goal()  # 다음 목표 전송

    def send_next_goal(self):
        if self.busy:
            self.get_logger().info("Robot is busy. Waiting for current goal to complete.")
            return

        if self.moving_path:
            next_goal = self.moving_path.pop(0)
            x, y, theta = next_goal
            self.busy = True  # 로봇 상태 설정
            self.send_goal(x, y, theta)
        else:
            self.get_logger().info("All goals have been reached.")

    def handle_table_request(self, msg):
        """
        테이블 요청 메시지 처리
        """
        self.get_logger().info(f"테이블 요청 수신: {msg.data}")  # 요청 수신 로그 출력
        self.request_queue.put(f"{msg.data}")  # 요청 내용을 Queue에 추가

    def handle_order(self, request, response):
        """
        주문 서비스 요청 처리
        """
        self.get_logger().info(f"주문 수신: 테이블 {request.table_num}, 메뉴 {request.menu}, 수량 {request.quantity}")
        self.request_queue.put(f"Table{request.table_num}: {request.menu} x {request.quantity}")  # 주문 내용을 Queue에 추가
        response.accept = True  # 주문을 수락
        self.get_logger().info("주문을 수락했습니다.")  # 로그 출력
        return response  # 응답 반환

    def send_confirm_request(self, menu):
        """
        확인 요청 전송
        """
        if not self.confirm_client.service_is_ready():
            self.get_logger().error("확인 서비스가 사용 불가능합니다!")  # 서비스 준비되지 않음
            return

        request = Confirm.Request()
        request.menu = menu  # 요청에 메뉴 이름 추가

        future = self.confirm_client.call_async(request)  # 비동기 요청
        rclpy.spin_until_future_complete(self, future)  # 응답 대기

        if future.result():
            self.get_logger().info(f"확인 응답: {future.result().accept}")  # 응답 결과 로그
        else:
            self.get_logger().error("확인 서비스 응답 실패!")  # 실패 시 로그

class StatsWindow(QDialog):
    """
    통계 데이터를 표시하는 창
    """
    def __init__(self, db_path, parent=None):
        super().__init__(parent)
        self.setWindowTitle("통계 열람")
        self.setFixedSize(1500, 1000)

        # SQLite 데이터베이스 연결
        self.db_connection = sqlite3.connect(db_path)
        self.db_cursor = self.db_connection.cursor()

        # 레이아웃 설정
        layout = QVBoxLayout(self)

        # 필터 옵션
        filter_layout = QHBoxLayout()
        self.date_label = QLabel("날짜별 조회:")
        self.date_input = QLineEdit()  # 날짜 입력 필드
        self.date_input.setPlaceholderText("YYYY-MM-DD")
        self.date_search_button = QPushButton("검색")
        self.date_search_button.clicked.connect(self.filter_by_date)

        self.time_label = QLabel("시간별 조회:")
        self.time_input = QLineEdit()  # 시간 입력 필드
        self.time_input.setPlaceholderText("HH:MM:SS")
        self.time_search_button = QPushButton("검색")
        self.time_search_button.clicked.connect(self.filter_by_time)

        filter_layout.addWidget(self.date_label)
        filter_layout.addWidget(self.date_input)
        filter_layout.addWidget(self.date_search_button)
        filter_layout.addWidget(self.time_label)
        filter_layout.addWidget(self.time_input)
        filter_layout.addWidget(self.time_search_button)

        layout.addLayout(filter_layout)

        # 데이터 테이블
        self.table_widget = QTableWidget()
        self.table_widget.setColumnCount(5)  # 열 개수
        self.table_widget.setHorizontalHeaderLabels(["메뉴 이름", "수량", "날짜", "시간", "총 가격"])
        layout.addWidget(self.table_widget)

        # 데이터 로드
        self.load_all_data()

    def load_all_data(self):
        """
        데이터베이스에서 모든 데이터를 로드하여 테이블에 표시
        """
        query = "SELECT menu_name, quantity, order_date, order_time, total_price FROM orders"
        self.populate_table(query)

    def filter_by_date(self):
        """
        입력된 날짜로 데이터를 필터링
        """
        date = self.date_input.text().strip()
        if date:
            query = f"SELECT menu_name, quantity, order_date, order_time, total_price FROM orders WHERE order_date = ?"
            self.populate_table(query, (date,))
        else:
            QMessageBox.warning(self, "경고", "날짜를 입력하세요.")

    def filter_by_time(self):
        """
        입력된 시간으로 데이터를 필터링
        """
        time = self.time_input.text().strip()
        if time:
            query = f"SELECT menu_name, quantity, order_date, order_time, total_price FROM orders WHERE order_time = ?"
            self.populate_table(query, (time,))
        else:
            QMessageBox.warning(self, "경고", "시간을 입력하세요.")

    def populate_table(self, query, params=()):
        """
        주어진 쿼리의 결과로 테이블을 채움
        """
        self.db_cursor.execute(query, params)
        results = self.db_cursor.fetchall()

        self.table_widget.setRowCount(len(results))  # 행 개수 설정
        for row_idx, row_data in enumerate(results):
            for col_idx, value in enumerate(row_data):
                self.table_widget.setItem(row_idx, col_idx, QTableWidgetItem(str(value)))

    def closeEvent(self, event):
        """
        창 닫힐 때 데이터베이스 연결 닫기
        """
        self.db_connection.close()
        event.accept()

class KitchenGUI(QMainWindow):
    """
    PyQt5 GUI: 주방 관리 UI
    """
    def __init__(self, ros_node, request_queue):
        super().__init__()
        self.ros_node = ros_node  # ROS2 노드 인스턴스
        self.request_queue = request_queue  # 요청 큐

        # DPI 스케일링 활성화
        QApplication.setAttribute(Qt.AA_EnableHighDpiScaling)
        QApplication.setAttribute(Qt.AA_UseHighDpiPixmaps)

        self.setWindowTitle("Kitchen GUI")  # 윈도우 제목 설정
        self.setGeometry(100, 100, 2560, 1280)  # 윈도우 크기 설정

        # SQLite3 데이터베이스 경로 설정
        self.db_path = "./src/restaurant.db"
        # ros_node.get_logger().info(f"Using database path: {db_path}")  # ROS2 노드 로거로 경로 출력
        
        # 데이터베이스 연결
        self.db_connection = sqlite3.connect(self.db_path) # 데이터베이스 연결
        self.db_cursor = self.db_connection.cursor()

        # 로봇 경로와 테이블 데이터 초기화
        self.robot_path = []  # 로봇 경로

        self.robot_path_num = [] # 전달할 로봇 경로
        self.table_coordinates = {
            1: (1.0, 2.0, 0.0),  # 테이블 1
            2: (3.0, 1.5, 0.0),  # 테이블 2
            3: (4.0, 3.0, 0.0),  # 테이블 3
            4: (0.0, 5.0, 0.0),  # 테이블 4
            5: (2.0, 2.0, 0.0),  # 테이블 5
            6: (3.5, 0.5, 0.0),  # 테이블 6
        }

        self.tables_data = {f"Table{i}": {} for i in range(1, 7)}  # 테이블별 데이터 저장

        # 메인 레이아웃 설정
        main_layout = QHBoxLayout()

        # 좌측 레이아웃: 테이블 및 로봇 경로 관리
        left_layout = QVBoxLayout()

        # 로봇 경로 텍스트박스
        self.robot_path_display = QTextEdit()
        self.robot_path_display.setReadOnly(True)  # 읽기 전용
        left_layout.addWidget(QLabel("로봇 경로"))
        left_layout.addWidget(self.robot_path_display, stretch=1)

        # 테이블 버튼 (6개)
        table_layout = QGridLayout()
        self.table_buttons = {}
        self.clear_buttons = {}  # 결제 버튼 저장

        for i in range(6):
            table_button = QPushButton(f"Table{i+1}\n\n")  # 테이블 정보 버튼
            table_button.setFixedSize(300, 300)  # 버튼 크기 설정
            table_button.clicked.connect(lambda _, t=f"Table{i+1}": self.toggle_table(t, i+1))  # 클릭 이벤트
            self.table_buttons[f"Table{i+1}"] = table_button  # 버튼 저장
            table_layout.addWidget(table_button, i // 2, i % 2)  # 2열 그리드 배치
        
             # 결제 버튼
            clear_button = QPushButton(f"결제 (Table{i+1})")  # 결제 버튼 생성
            clear_button.setFixedSize(300, 50)  # 결제 버튼 크기 설정
            clear_button.clicked.connect(lambda _, t=f"Table{i+1}": self.clear_table_data(t))  # 클릭 이벤트
            self.clear_buttons[f"Table{i+1}"] = clear_button  # 결제 버튼 저장
            table_layout.addWidget(clear_button, i // 2 + 3, i % 2)  # 결제 버튼 추가
        left_layout.addLayout(table_layout, stretch=9)

        # 좌측 레이아웃을 메인 레이아웃에 추가 (50% 비율)
        main_layout.addLayout(left_layout, 5)

        # 중앙 레이아웃: 요청 및 메뉴 접수
        center_layout = QVBoxLayout()

        # 요청 리스트
        self.request_list = QListWidget()
        self.request_list.itemClicked.connect(self.handle_request_completion)  # 요청 완료 처리
        center_layout.addWidget(QLabel("요청 사항"))
        center_layout.addWidget(self.request_list, stretch=1)

        # 메뉴 접수 리스트
        self.menu_list = QListWidget()
        self.menu_list.itemClicked.connect(self.handle_menu_selection)  # 메뉴 선택 처리
        center_layout.addWidget(QLabel("메뉴 접수"))
        center_layout.addWidget(self.menu_list, stretch=2)

        # 중앙 레이아웃을 메인 레이아웃에 추가 (40% 비율)
        main_layout.addLayout(center_layout, 4)

        # 우측 레이아웃: 통계 및 로봇 제어
        right_layout = QVBoxLayout()
        right_layout.setAlignment(Qt.AlignCenter)
        # 버튼 간 간격 추가
        button_spacing = 20

        # 통계 버튼
        stats_button = QPushButton("통계")
        stats_button.setFixedSize(120, 40)  # 크기 설정
        stats_button.clicked.connect(self.open_stats_window)  # 통계 창 연결
        right_layout.addWidget(stats_button)
        right_layout.addSpacing(button_spacing)

        # 편집 버튼
        edit_button = QPushButton("편집")
        edit_button.setFixedSize(120, 40)
        edit_button.clicked.connect(self.edit_table_menu)
        right_layout.addWidget(edit_button)
        right_layout.addSpacing(button_spacing)

        # 호출 버튼
        self.robot_call_button = QPushButton("로봇 호출")
        self.robot_call_button.clicked.connect(self.call_robot)
        self.robot_call_button.setFixedSize(120, 40)
        right_layout.addWidget(self.robot_call_button)
        right_layout.addSpacing(button_spacing)

        # 로봇 출발 버튼
        self.robot_start_button = QPushButton("로봇 출발")
        self.robot_start_button.clicked.connect(self.send_path_robot)
        self.robot_start_button.setFixedSize(120, 40)
        right_layout.addWidget(self.robot_start_button)

        # 우측 레이아웃을 메인 레이아웃에 추가 (10% 비율)
        main_layout.addLayout(right_layout, 1)

        # 메인 레이아웃을 중앙 위젯에 적용
        central_widget = QWidget()
        central_widget.setLayout(main_layout)
        self.setCentralWidget(central_widget)

        # QTimer를 사용해 요청 확인
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.check_requests)  # 요청 확인
        self.timer.start(100)  # 100ms마다 실행

    def call_robot(self):
        self.ros_node.moving_path.append((0, 0, 0))  # KitchenNode의 moving_path에 전달
        self.ros_node.send_next_goal()

    def send_path_robot(self):
        self.moving_path.extend(self.robot_path_num)
        self.robot_path_num=[]
        self.robot_path=[]
        self.update_robot_path_display()

    def send_path_robot(self):
        if not self.robot_path_num:
            self.ros_node.get_logger().warn("No waypoints to send.")
            return

        # 로봇 경로를 moving_path로 전달
        for num in self.robot_path_num:
            x,y,theta = self.table_coordinates[num]
            self.ros_node.moving_path.append((x, y, theta))  # KitchenNode의 moving_path에 전달

        self.robot_path_num = []  # 초기화
        self.robot_path = []
        self.update_robot_path_display()

        # 첫 번째 목표 전송
        self.ros_node.send_next_goal()

    def open_stats_window(self):
        """
        통계 창 열기
        """
        stats_window = StatsWindow(self.db_path, self)
        stats_window.exec_()

    def save_order_to_db(self, menu_name, quantity):
        """
        SQLite3 데이터베이스에 주문 저장
        """
        global menu

        now = datetime.now()  # 현재 날짜와 시간 가져오기
        order_date = now.strftime("%Y-%m-%d")  # 날짜 포맷
        order_time = now.strftime("%H:%M:%S")  # 시간 포맷
        total_price = quantity * menu[menu_name]["price"]

        # 데이터베이스에 삽입
        self.db_cursor.execute("""
            INSERT INTO orders (menu_name, quantity, order_date, order_time, total_price)
            VALUES (?, ?, ?, ?, ?);
        """, (menu_name, quantity, order_date, order_time, total_price))
        self.db_connection.commit()  # 변경 사항 저장

    def toggle_table(self, table_name, table_num):
        """테이블 클릭 시 로봇 경로 추가/삭제"""
        if table_name in self.robot_path:
            self.robot_path.remove(table_name)  # 경로에서 제거
            self.robot_path_num.remove(table_num)
        else:
            self.robot_path.append(table_name)  # 경로에 추가
            self.robot_path_num.append(table_num)
        self.update_robot_path_display()


    def update_robot_path_display(self):
        """로봇 경로 실시간 업데이트"""
        self.robot_path_display.setText(" -> ".join(self.robot_path))  # 경로 표시

    def handle_request_completion(self, item):
        """요청 사항 완료 처리"""
        if item:
            reply = QMessageBox.question(
                self, "요청 완료", f"{item.text()}를 완료하시겠습니까?",
                QMessageBox.Yes | QMessageBox.No
            )
            if reply == QMessageBox.Yes:
                self.request_list.takeItem(self.request_list.row(item))  # 요청 리스트에서 제거

    def handle_menu_selection(self, item):
        """메뉴 접수/거절 처리"""
        if item:
            dialog = QDialog(self)
            dialog.setWindowTitle("메뉴 처리")
            layout = QVBoxLayout(dialog)

            label = QLabel(f"{item.text()}를 어떻게 처리하시겠습니까?")
            layout.addWidget(label)

            accept_button = QPushButton("접수")
            accept_button.clicked.connect(lambda: self.process_menu(item, True, dialog))
            
            reject_button = QPushButton("거절")
            reject_button.clicked.connect(lambda: self.process_menu(item, False, dialog))

            layout.addWidget(accept_button)
            layout.addWidget(reject_button)
            dialog.setLayout(layout)
            dialog.exec_()

    def process_menu(self, item, action, dialog):
        """
        메뉴를 접수 또는 거절
        """
        if action:
            table_name, menu_data = item.text().split(": ", 1)
            menu_name, quantity_str = menu_data.split(" x ")
            quantity = int(quantity_str)  # 수량 변환
            
            # 기존 메뉴가 있는지 확인하고 수량 합산
            if menu_name in self.tables_data[table_name]:
                self.tables_data[table_name][menu_name] += quantity
            else:
                self.tables_data[table_name][menu_name] = quantity
            
            # 테이블 UI 업데이트
            self.update_table_display(table_name)

            # SQLite3에 주문 저장
            self.save_order_to_db(menu_name, quantity)

        self.menu_list.takeItem(self.menu_list.row(item))
        dialog.accept()

    def closeEvent(self, event):
        """
        GUI 종료 시 데이터베이스 연결 닫기
        """
        self.db_connection.close()
        event.accept()

    def update_table_display(self, table_name):
        """
        테이블 메뉴 정보 업데이트
        """
        menu_info = "\n".join(
            f"{menu_name} - 수량: {quantity}"
            for menu_name, quantity in self.tables_data[table_name].items()
        )
        self.table_buttons[table_name].setText(f"{table_name}\n\n{menu_info}")

    def check_requests(self):
        """Queue에서 요청 확인"""
        while not self.request_queue.empty():
            request = self.request_queue.get()
            if "Table 요청" in request:
                self.request_list.addItem(request)  # 요청 리스트에 추가
            else:
                self.menu_list.addItem(request)  # 메뉴 리스트에 추가

    def clear_table_data(self, table_name):
        """
        특정 테이블 데이터를 초기화하고 결제 금액을 보여주는 팝업창을 표시
        """
        # 테이블에 있는 메뉴들의 총 금액 계산
        total_amount = 0
        for menu_name, quantity in self.tables_data[table_name].items():
            total_amount += menu[menu_name]["price"] * quantity  # 가격 * 수량

        # 결제 금액 팝업창 표시
        reply = QMessageBox.question(
            self, "결제", f"{table_name}의 결제 금액은 {total_amount}원입니다. 결제를 완료하시겠습니까?",
            QMessageBox.Yes | QMessageBox.No
        )

        # 결제 완료 시 데이터 초기화
        if reply == QMessageBox.Yes:
            self.tables_data[table_name] = {}  # 데이터 초기화
            self.update_table_display(table_name)  # UI 업데이트
            QMessageBox.information(self, "완료", f"{table_name}의 결제가 완료되었습니다.")

    def edit_table_menu(self):
        """
        테이블 메뉴 편집: 이동할 소스 테이블과 대상 테이블을 선택
        """
        dialog = QDialog(self)
        dialog.setWindowTitle("테이블 메뉴 편집")
        dialog_layout = QVBoxLayout(dialog)

        label = QLabel("먼저 이동할 메뉴가 있는 테이블을 선택하세요.")
        dialog_layout.addWidget(label)

        # 소스 테이블 버튼 생성
        for table_name in self.tables_data.keys():
            button = QPushButton(f"{table_name}")
            button.clicked.connect(lambda _, t=table_name, d=dialog: self.select_source_table(t, d))
            dialog_layout.addWidget(button)

        dialog.setLayout(dialog_layout)
        dialog.exec_()

    def select_source_table(self, source_table, dialog):
        """
        소스 테이블 선택 후 대상 테이블 선택 다이얼로그 열기
        """
        if not self.tables_data[source_table]:  # 소스 테이블에 메뉴가 없을 경우
            QMessageBox.warning(self, "오류", f"{source_table}에는 메뉴가 없습니다.")
            return

        # 대상 테이블 선택 다이얼로그 생성
        target_dialog = QDialog(self)
        target_dialog.setWindowTitle("대상 테이블 선택")
        target_dialog_layout = QVBoxLayout(target_dialog)

        label = QLabel(f"{source_table}에서 메뉴를 이동할 대상 테이블을 선택하세요.")
        target_dialog_layout.addWidget(label)

        # 대상 테이블 버튼 생성
        for table_name in self.tables_data.keys():
            if table_name != source_table:  # 소스 테이블과 다른 테이블만 표시
                button = QPushButton(f"{table_name}")
                button.clicked.connect(lambda _, s=source_table, t=table_name, d=target_dialog: self.move_menu_to_table(s, t, d))
                target_dialog_layout.addWidget(button)

        target_dialog.setLayout(target_dialog_layout)
        target_dialog.exec_()
    
    def move_menu_to_table(self, source_table, target_table, dialog):
        """
        소스 테이블에서 대상 테이블로 메뉴 이동
        """
        # 메뉴 데이터를 합산하여 이동
        for menu_name, quantity in self.tables_data[source_table].items():
            if menu_name in self.tables_data[target_table]:
                self.tables_data[target_table][menu_name] += quantity
            else:
                self.tables_data[target_table][menu_name] = quantity

        # 소스 테이블 데이터 초기화
        self.tables_data[source_table] = {}

        # UI 업데이트
        self.update_table_display(source_table)
        self.update_table_display(target_table)

        QMessageBox.information(self, "완료", f"{source_table}의 메뉴가 {target_table}로 이동되었습니다.")
        dialog.accept()


def main():
    """
    메인 실행 함수
    """
    rclpy.init()  # ROS2 초기화
    request_queue = queue.Queue()  # 요청 큐 생성
    ros_node = KitchenNode(request_queue)  # ROS 노드 생성

    # ROS2 스레드 관리
    running = threading.Event()
    running.set()

    def ros_spin():
        while running.is_set():
            rclpy.spin_once(ros_node, timeout_sec=0.1)  # ROS 메시지 처리

    ros_thread = threading.Thread(target=ros_spin, daemon=True)  # ROS 스레드 생성
    ros_thread.start()

    # PyQt5 실행
    app = QApplication(sys.argv)
    gui = KitchenGUI(ros_node, request_queue)
    gui.show()

    try:
        sys.exit(app.exec_())  # PyQt 애플리케이션 실행
    except KeyboardInterrupt:
        print("종료 중...")
    finally:
        running.clear()  # 스레드 종료
        ros_thread.join()
        ros_node.destroy_node()  # 노드 종료
        rclpy.shutdown()  # ROS2 종료

if __name__ == "__main__":
    main()


In [6]:
# create sql db
import sqlite3

# 데이터베이스 연결 (파일로 저장되며, 파일 이름은 예시로 'example.db')
conn = sqlite3.connect('restaurant.db')
cursor = conn.cursor()


create_table_query = '''
CREATE TABLE orders (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    menu_name TEXT NOT NULL,
    quantity INTEGER NOT NULL,
    order_date DATE NOT NULL,
    order_time TIME NOT NULL,
    total_price INTEGER NOT NULL
);
'''

# 테이블 생성
cursor.execute(create_table_query)

# 변경 사항 저장
conn.commit()

In [ ]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
from srv_interface.srv import Order
from rclpy.qos import QoSProfile

import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QListWidget, QStackedWidget, QPushButton,
    QVBoxLayout, QLabel, QDialog, QHBoxLayout, QWidget, QGridLayout, QScrollArea,
    QListWidgetItem, QMessageBox
)
from PyQt5.QtGui import QFont, QPixmap
from PyQt5.QtCore import Qt


class TableMonitorNode(Node):
    """
    ROS2 노드: 테이블 모니터 역할을 수행합니다.
    - 고객의 요청 사항을 퍼블리시합니다.
    - 주방에 주문을 서비스 요청합니다.
    - 배달 확인 요청에 대한 서비스를 제공합니다.
    """
    def __init__(self):
        super().__init__('table_monitor')  # 노드 이름 설정
        qos_profile = QoSProfile(depth=10)  # QoS 설정

        # Publisher: 고객 요청사항을 'table_request' 토픽에 발행
        self.publisher = self.create_publisher(String, 'table_request', qos_profile)

        # Client: 'kitchen_order' 서비스에 주문 요청
        self.request_client = self.create_client(Order, 'kitchen_order')

        self.get_logger().info("Table Monitor가 실행 중입니다")

    def publish_request(self, request_type):
        """
        고객 요청사항을 'table_request' 토픽에 발행합니다.
        :param request_type: 고객이 요청한 서비스 (예: '물', '앞치마' 등)
        """
        msg = String()
        msg.data = request_type  # 요청 내용을 메시지에 담기
        self.publisher.publish(msg)  # 메시지 발행
        self.get_logger().info(f"고객 요청사항 발행: {request_type}")

    def send_order_request(self, table_num, menu, quantity):
        """
        주방에 주문 요청을 전송합니다.
        :param table_num: 테이블 번호
        :param menu: 주문한 메뉴 이름
        :param quantity: 주문 수량
        """
        # 주문 서비스가 사용 가능한지 확인
        if not self.request_client.wait_for_service(timeout_sec=3.0):
            self.get_logger().error("주문 서비스가 현재 사용 불가능합니다!")
            return

        # 주문 요청 생성
        request = Order.Request()
        request.table_num = table_num
        request.menu = menu
        request.quantity = quantity

        # 주문 서비스 호출
        future = self.request_client.call_async(request)
        rclpy.spin_until_future_complete(self, future)  # 응답 대기

        # 응답 처리
        if future.result():
            response = future.result()
            if response.accept:
                self.get_logger().info(f"주문 접수 완료: {menu} x {quantity}")
            else:
                self.get_logger().warning("주문이 거절되었습니다!")
        else:
            self.get_logger().error("서비스 응답 실패!")

    def handle_response(self, request, response):
        """
        배달 확인 요청에 응답합니다.
        :param request: 배달 확인 요청 (메뉴 정보 포함)
        :param response: 배달 확인 응답 (수락 여부)
        """
        self.get_logger().info(f"배달 확인 요청: {request.menu}")
        response.accept = True  # 배달을 수락으로 설정
        self.get_logger().info("배달 확인 응답 전송 완료")
        return response  # 응답 반환


class MainWindow(QMainWindow):
    """
    PyQt5 기반의 GUI 클래스: 테이블 모니터 역할을 수행합니다.
    - 메뉴 선택 및 주문 기능을 제공합니다.
    - 요청 사항을 전송할 수 있습니다.
    - 장바구니를 관리하고 결제 기능을 제공합니다.
    """
    def __init__(self, ros_node):
        super().__init__()
        self.cart = []  # 장바구니 데이터 저장
        self.table_num = 1  # 테이블 번호 (고정값으로 설정되어 있으나 필요에 따라 변경 가능)
        self.ros_node = ros_node  # ROS2 노드 연결
        
        # GUI 설정
        self.setWindowTitle("Restaurant Serving Robot")  # 윈도우 제목 설정
        self.setGeometry(100, 100, 2560, 1280)  # 윈도우 크기 및 위치 설정

        # 중앙 위젯과 레이아웃 설정
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        main_layout = QHBoxLayout(self.central_widget)  # 전체 수평 레이아웃 설정

        # 왼쪽 영역: 메뉴 카테고리 및 요청사항 버튼
        left_layout = QVBoxLayout()
        self.menu_list = QListWidget()  # 메뉴 카테고리 리스트 위젯
        self.menu_list.addItems(["메인", "음료", "주류"])  # 카테고리 추가
        self.menu_list.itemClicked.connect(self.change_menu)  # 카테고리 선택 시 이벤트 연결
        left_layout.addWidget(self.menu_list)

        self.request_button = QPushButton("요청사항")  # 요청사항 버튼
        self.request_button.clicked.connect(self.open_request_dialog)  # 클릭 시 이벤트 연결
        left_layout.addWidget(self.request_button)

        left_container = QWidget()
        left_container.setLayout(left_layout)
        main_layout.addWidget(left_container, stretch=1)  # 왼쪽 영역을 메인 레이아웃에 추가

        # 중앙 영역: 메뉴 선택 영역
        self.menu_stacked_widget = QStackedWidget()  # 스택 위젯을 사용하여 메뉴 페이지 전환
        self.setup_menu_pages()  # 메뉴 페이지 설정
        main_layout.addWidget(self.menu_stacked_widget, stretch=6)  # 중앙 영역을 메인 레이아웃에 추가

        # 오른쪽 영역: 장바구니 및 결제 버튼
        right_layout = QVBoxLayout()
        self.cart_widget = QListWidget()  # 장바구니 리스트 위젯
        self.cart_widget.itemClicked.connect(self.confirm_cancel)  # 장바구니 항목 클릭 시 취소 확인
        right_layout.addWidget(self.cart_widget)

        self.total_price_label = QLabel("총 가격: 0원")  # 총 가격 레이블 추가
        self.total_price_label.setFont(QFont("Arial", 12))  # 폰트 설정
        right_layout.addWidget(self.total_price_label)

        self.checkout_button = QPushButton("접수")  # 접수 버튼
        self.checkout_button.clicked.connect(self.checkout)  # 클릭 시 이벤트 연결
        right_layout.addWidget(self.checkout_button)

        right_container = QWidget()
        right_container.setLayout(right_layout)
        main_layout.addWidget(right_container, stretch=2)  # 오른쪽 영역을 메인 레이아웃에 추가

    def setup_menu_pages(self):
        """
        각 메뉴 카테고리에 대한 페이지를 설정합니다.
        """
        # 메뉴 데이터 정의 (이미지 파일 경로 포함)
        menu = {
            "메인": [
                {"name": "치킨","price": 18000, "image": "./src/images/chicken.jpg"},
                {"name": "피자", "price": 28000, "image": "./src/images/pizza.jpeg"},
                {"name": "햄버거", "price": 9000, "image": "./src/images/burger.jpg"},
                {"name": "스테이크", "price": 20000, "image": "./src/images/steak.jpg"},
                {"name": "파스타", "price": 12000, "image": "./src/images/pasta.jpeg"},
                {"name": "라자냐", "price": 11000, "image": "./src/images/lasagna.jpg"},
                {"name": "비프 웰링턴","price": 65000, "image": "./src/images/beef_wellington.jpg"},
            ],
            "음료": [
                {"name": "제로 콜라", "price": 2000, "image": "./src/images/cola.jpg"},
                {"name": "스프라이트", "price": 2000, "image": "./src/images/sprite.png"},
                {"name": "환타", "price": 2000, "image": "./src/images/fanta.png"},
            ],
            "주류": [
                {"name": "하이볼", "price": 5000, "image": "./src/images/highball.jpeg"},
                {"name": "생맥주 500cc", "price": 5000, "image": "./src/images/beer.jpg"},
                {"name": "소주", "price": 6000, "image": "./src/images/soju.png"},
            ],
        }

        self.pages = {}  # 카테고리별 페이지를 저장할 딕셔너리

        for category, items in menu.items():
            # 스크롤 영역 생성
            scroll_area = QScrollArea()
            scroll_area.setWidgetResizable(True)  # 내용 크기에 따라 스크롤 조정
            
            page = QWidget()
            page_layout = QGridLayout(page)  # 2열 그리드 레이아웃
            page_layout.setAlignment(Qt.AlignTop)  # 위쪽 정렬

            for i, item in enumerate(items):
                # 이미지 생성
                image_label = QLabel()
                pixmap = QPixmap(item["image"])  # 이미지 로드
                pixmap = pixmap.scaled(500, 400, Qt.IgnoreAspectRatio)  # 크기 조정
                image_label.setPixmap(pixmap)

                # 버튼 생성
                menu_button = QPushButton(f'{item["name"]}\n가격: {item["price"]}')
                menu_button.setFixedSize(500, 100)  # 버튼 크기 설정
                menu_button.clicked.connect(lambda _, name=item["name"], price=item["price"]: self.open_quantity_dialog(name, price))

                # 이미지와 버튼을 수직으로 정렬
                container = QVBoxLayout()
                container.addWidget(image_label)
                container.addWidget(menu_button)

                # 컨테이너를 위젯으로 감싸고 그리드 레이아웃에 추가
                widget = QWidget()
                widget.setLayout(container)
                page_layout.addWidget(widget, i // 2, i % 2, alignment=Qt.AlignCenter)  # 2열로 배치

            # 스크롤 영역에 페이지 설정
            page.setLayout(page_layout)
            scroll_area.setWidget(page)

            # 페이지를 스택 위젯에 추가
            self.menu_stacked_widget.addWidget(scroll_area)
            self.pages[category] = scroll_area

    def change_menu(self, item):
        """
        메뉴 카테고리를 변경합니다.
        :param item: 선택된 QListWidgetItem (카테고리 이름)
        """
        category = item.text()
        if category in self.pages:
            self.menu_stacked_widget.setCurrentWidget(self.pages[category])  # 선택된 카테고리의 페이지로 전환

    def open_quantity_dialog(self, menu_name, price):
        """
        메뉴 수량을 결정하는 다이얼로그를 엽니다.
        :param menu_name: 선택된 메뉴의 이름
        """
        dialog = QDialog(self)
        dialog.setWindowTitle(f"{menu_name} 수량 결정")
        
        # 팝업창 크기 설정
        dialog.setFixedSize(400, 300)  # 팝업창 크기를 가로 400px, 세로 300px로 설정
        layout = QVBoxLayout(dialog)

        quantity_label = QLabel("수량: 1")  # 초기 수량은 1로 설정
        layout.addWidget(quantity_label)
        count = 1  # 수량을 저장할 변수

        # 수량 증가 함수
        def increase_quantity():
            nonlocal count
            count += 1
            quantity_label.setText(f"수량: {count}")

        # 수량 감소 함수
        def decrease_quantity():
            nonlocal count
            if count > 1:
                count -= 1
                quantity_label.setText(f"수량: {count}")

        # 수량 조절 버튼 생성
        plus_button = QPushButton("+")
        plus_button.clicked.connect(increase_quantity)
        minus_button = QPushButton("-")
        minus_button.clicked.connect(decrease_quantity)
        layout.addWidget(plus_button)
        layout.addWidget(minus_button)

        # 확인 버튼: 장바구니에 메뉴 추가
        confirm_button = QPushButton("확인")
        confirm_button.clicked.connect(lambda: [self.add_to_cart(menu_name, count, count * price), dialog.accept()])
        layout.addWidget(confirm_button)

        dialog.setLayout(layout)
        dialog.exec_()  # 다이얼로그 실행

    def add_to_cart(self, menu_name, count, money):
        """
        선택된 메뉴와 수량을 장바구니에 추가합니다.
        :param menu_name: 메뉴 이름
        :param count: 선택된 수량
        """
        for item in self.cart:
            if item['menu'] == menu_name:
                item['quantity'] += count  # 수량 업데이트
                item['all_price'] += money
                break
        else:
            self.cart.append({"menu": menu_name, "quantity": count, "all_price": money})
        # 장바구니 UI 업데이트
        self.update_cart_view()

    def confirm_cancel(self, item):
        """
        장바구니 항목 클릭 시 취소 여부를 묻는 다이얼로그를 띄웁니다.
        :param item: 클릭된 장바구니 항목
        """
        # 취소 여부를 묻는 메시지 박스 생성
        reply = QMessageBox.question(self, '취소 확인', f"{item.text()} 메뉴를 장바구니에서 삭제하시겠습니까?",
                                     QMessageBox.Yes | QMessageBox.No, QMessageBox.No)

        if reply == QMessageBox.Yes:
            self.remove_from_cart(item)  # 장바구니에서 항목 제거

    def remove_from_cart(self, item):
        """
        장바구니에서 메뉴 항목을 제거합니다.
        :param item: 제거할 장바구니 항목
        """
        cart_item = item.data(Qt.UserRole)  # 장바구니 아이템 데이터 가져오기
        self.cart.remove(cart_item)  # 장바구니에서 항목 제거
        self.update_cart_view()  # 장바구니 UI 업데이트

    def update_cart_view(self):
        """
        장바구니 리스트 위젯을 업데이트합니다.
        """
        self.cart_widget.clear()  # 기존 항목 제거
        total_price = 0  # 총 가격 변수 초기화
        for item in self.cart:
            cart_item_widget = QListWidgetItem(f"{item['menu']} - 수량: {item['quantity']} - 총 가격: {item['all_price']}")  # 장바구니 아이템 표시
            cart_item_widget.setData(Qt.UserRole, item)  # 아이템 데이터를 저장 (수정할 때 사용)
            self.cart_widget.addItem(cart_item_widget)
            total_price += item['all_price']  # 총 가격 업데이트

        self.total_price_label.setText(f"총 가격: {total_price}원")  # 총 가격 레이블 업데이트

    def checkout(self):
        """
        결제 버튼 클릭 시 호출됩니다.
        장바구니에 있는 모든 메뉴를 주문 요청으로 전송합니다.
        """
        for item in self.cart:
            self.ros_node.send_order_request(self.table_num, item['menu'], item['quantity'])  # 주문 요청 전송
        self.cart.clear()  # 장바구니 비우기
        self.update_cart_view()  # 장바구니 뷰 업데이트

    def open_request_dialog(self):
        """
        고객 요청사항을 선택할 수 있는 다이얼로그를 엽니다.
        """
        dialog = QDialog(self)
        dialog.setWindowTitle("요청사항")
        layout = QVBoxLayout(dialog)

        # 요청사항 목록
        requests = ["물", "앞치마", "앞접시", "물티슈", "숟가락 & 젓가락", "직원 호출"]
        active_requests = set()  # 선택된 요청사항을 저장할 집합

        # 요청사항 버튼 생성 및 클릭 이벤트 연결
        def toggle_request(button, request):
            if request in active_requests:
                active_requests.remove(request)
                button.setStyleSheet("background-color: none;")  # 선택 해제 시 배경색 제거
            else:
                active_requests.add(request)
                button.setStyleSheet("background-color: lightblue;")  # 선택 시 배경색 변경

        for request in requests:
            button = QPushButton(request)
            button.setCheckable(True)
            button.clicked.connect(lambda _, b=button, r=request: toggle_request(b, r))
            layout.addWidget(button)

        # 확인 버튼: 선택된 요청사항을 퍼블리시
        confirm_button = QPushButton("확인")
        confirm_button.clicked.connect(lambda: self.confirm_requests(dialog, active_requests))
        layout.addWidget(confirm_button)

        dialog.setLayout(layout)
        dialog.exec_()  # 다이얼로그 실행

    def confirm_requests(self, dialog, active_requests):
        """
        선택된 요청사항을 퍼블리시합니다.
        :param dialog: 현재 다이얼로그 객체
        :param active_requests: 선택된 요청사항 집합
        """
        for request in active_requests:
            message = f"{self.table_num}번 Table 요청: {request}"
            self.ros_node.publish_request(message)  # 요청사항 퍼블리시
        dialog.accept()  # 다이얼로그 닫기


def main(args=None):
    """
    프로그램의 진입점 함수입니다.
    ROS2 노드를 초기화하고, PyQt5 애플리케이션을 실행합니다.
    """
    rclpy.init(args=args)  # ROS2 초기화
    ros_node = TableMonitorNode()  # ROS2 노드 생성

    app = QApplication(sys.argv)  # PyQt5 애플리케이션 생성
    main_window = MainWindow(ros_node)  # 메인 윈도우 생성
    main_window.show()  # 윈도우 표시

    app.exec_()  # 이벤트 루프 실행
    ros_node.destroy_node()  # ROS2 노드 종료
    rclpy.shutdown()  # ROS2 종료


if __name__ == "__main__":
    main()  # 프로그램 실행


In [ ]:
import sys
import threading
import queue
import rclpy
from rclpy.node import Node
from rclpy.qos import QoSProfile
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QPushButton, QVBoxLayout, QHBoxLayout,
    QGridLayout, QLabel, QListWidget, QWidget, QTextEdit, QDialog, QMessageBox, 
    QLineEdit, QTableWidget, QTableWidgetItem
)
from PyQt5.QtCore import Qt, QTimer
from std_msgs.msg import String
from srv_interface.srv import Order
import sqlite3
from datetime import datetime

from nav2_msgs.action import NavigateToPose
from rclpy.action import ActionClient

menu = {
        "치킨": {"price": 18000, "image": "./src/images/chicken.jpg"},
        "피자": {"price": 28000, "image": "./src/images/pizza.jpeg"},
        "햄버거": {"price": 9000, "image": "./src/images/burger.jpg"},
        "스테이크": {"price": 20000, "image": "./src/images/steak.jpg"},
        "파스타": {"price": 12000, "image": "./src/images/pasta.jpeg"},
        "라자냐": {"price": 11000, "image": "./src/images/lasagna.jpg"},
        "비프 웰링턴": {"price": 65000, "image": "./src/images/beef_wellington.jpg"},
        "제로 콜라": {"price": 2000, "image": "./src/images/cola.jpg"},
        "스프라이트": {"price": 2000, "image": "./src/images/sprite.png"},
        "환타": {"price": 2000, "image": "./src/images/fanta.png"},
        "하이볼": {"price": 5000, "image": "./src/images/highball.jpeg"},
        "생맥주 500cc": {"price": 5000, "image": "./src/images/beer.jpg"},
        "소주": {"price": 6000, "image": "./src/images/soju.png"},
        }

class KitchenNode(Node):
    """
    ROS2 Node: 주방 기능 처리
    """
    def __init__(self, request_queue):
        super().__init__('kitchen_node')  # 노드 이름 설정
        self.request_queue = request_queue  # 메시지를 전달받을 큐 생성
        qos_profile = QoSProfile(depth=10)  # QoS 설정

        # 테이블 요청 구독: 'table_request' 토픽에서 메시지를 받아 처리
        self.table_request_sub = self.create_subscription(
            String, 'table_request', self.handle_table_request, qos_profile)

        # 주문 서비스 서버: 'kitchen_order' 서비스 요청을 받아 처리
        self.order_service = self.create_service(Order, 'kitchen_order', self.handle_order)

        self._action_client = ActionClient(self, NavigateToPose, 'navigate_to_pose')
        self.moving_path = []  # 경로를 저장하는 리스트
        self.busy = False  # 로봇 상태 플래그

        self.get_logger().info("KitchenNode가 초기화되었습니다.")

    def send_goal(self, x, y, theta):
        goal_msg = NavigateToPose.Goal()
        goal_msg.pose.header.frame_id = "map"
        goal_msg.pose.header.stamp = self.get_clock().now().to_msg()
        goal_msg.pose.pose.position.x = x
        goal_msg.pose.pose.position.y = y
        goal_msg.pose.pose.orientation.z = theta
        goal_msg.pose.pose.orientation.w = 1.0
        self._action_client.wait_for_server()
        self.get_logger().info(f'Sending goal to: x={x}, y={y}, theta={theta}')
        self._action_client.send_goal_async(
            goal_msg,
            feedback_callback=self.feedback_callback
        ).add_done_callback(self.goal_response_callback)

    def feedback_callback(self, feedback_msg):
        feedback = feedback_msg.feedback
        self.get_logger().info(f"Current progress: {feedback.current_pose}")

    def goal_response_callback(self, future):
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().info('Goal rejected :(')
            return
        self.get_logger().info('Goal accepted :)')
        goal_handle.get_result_async().add_done_callback(self.get_result_callback)

    def get_result_callback(self, future):
        result = future.result().result
        self.get_logger().info(f'Result: {result}')
        self.busy = False  # 목표 완료, 상태 해제
        self.send_next_goal()  # 다음 목표 전송

    def send_next_goal(self):
        if self.busy:
            self.get_logger().info("Robot is busy. Waiting for current goal to complete.")
            return

        if self.moving_path:
            next_goal = self.moving_path.pop(0)
            x, y, theta = next_goal
            self.busy = True  # 로봇 상태 설정
            self.send_goal(x, y, theta)
        else:
            self.get_logger().info("All goals have been reached.")

    def handle_table_request(self, msg):
        """
        테이블 요청 메시지 처리
        """
        self.get_logger().info(f"테이블 요청 수신: {msg.data}")  # 요청 수신 로그 출력
        self.request_queue.put(f"{msg.data}")  # 요청 내용을 Queue에 추가

    def handle_order(self, request, response):
        """
        주문 서비스 요청 처리
        """
        self.get_logger().info(f"주문 수신: 테이블 {request.table_num}, 메뉴 {request.menu}, 수량 {request.quantity}")
        self.request_queue.put(f"Table{request.table_num}: {request.menu} x {request.quantity}")  # 주문 내용을 Queue에 추가
        response.accept = True  # 주문을 수락
        self.get_logger().info("주문을 수락했습니다.")  # 로그 출력
        return response  # 응답 반환

class StatsWindow(QDialog):
    """
    통계 데이터를 표시하는 창
    """
    def __init__(self, db_path, parent=None):
        super().__init__(parent)
        self.setWindowTitle("통계 열람")
        self.setFixedSize(1500, 1000)

        # SQLite 데이터베이스 연결
        self.db_connection = sqlite3.connect(db_path)
        self.db_cursor = self.db_connection.cursor()

        # 레이아웃 설정
        layout = QVBoxLayout(self)

        # 필터 옵션
        filter_layout = QHBoxLayout()
        self.date_label = QLabel("날짜별 조회:")
        self.date_input = QLineEdit()  # 날짜 입력 필드
        self.date_input.setPlaceholderText("YYYY-MM-DD")
        self.date_search_button = QPushButton("검색")
        self.date_search_button.clicked.connect(self.filter_by_date)

        self.time_label = QLabel("시간별 조회:")
        self.time_input = QLineEdit()  # 시간 입력 필드
        self.time_input.setPlaceholderText("HH:MM:SS")
        self.time_search_button = QPushButton("검색")
        self.time_search_button.clicked.connect(self.filter_by_time)

        filter_layout.addWidget(self.date_label)
        filter_layout.addWidget(self.date_input)
        filter_layout.addWidget(self.date_search_button)
        filter_layout.addWidget(self.time_label)
        filter_layout.addWidget(self.time_input)
        filter_layout.addWidget(self.time_search_button)

        layout.addLayout(filter_layout)

        # 데이터 테이블
        self.table_widget = QTableWidget()
        self.table_widget.setColumnCount(5)  # 열 개수
        self.table_widget.setHorizontalHeaderLabels(["메뉴 이름", "수량", "날짜", "시간", "총 가격"])
        layout.addWidget(self.table_widget)

        # 데이터 로드
        self.load_all_data()

    def load_all_data(self):
        """
        데이터베이스에서 모든 데이터를 로드하여 테이블에 표시
        """
        query = "SELECT menu_name, quantity, order_date, order_time, total_price FROM orders"
        self.populate_table(query)

    def filter_by_date(self):
        """
        입력된 날짜로 데이터를 필터링
        """
        date = self.date_input.text().strip()
        if date:
            query = f"SELECT menu_name, quantity, order_date, order_time, total_price FROM orders WHERE order_date = ?"
            self.populate_table(query, (date,))
        else:
            QMessageBox.warning(self, "경고", "날짜를 입력하세요.")

    def filter_by_time(self):
        """
        입력된 시간으로 데이터를 필터링
        """
        time = self.time_input.text().strip()
        if time:
            query = f"SELECT menu_name, quantity, order_date, order_time, total_price FROM orders WHERE order_time = ?"
            self.populate_table(query, (time,))
        else:
            QMessageBox.warning(self, "경고", "시간을 입력하세요.")

    def populate_table(self, query, params=()):
        """
        주어진 쿼리의 결과로 테이블을 채움
        """
        self.db_cursor.execute(query, params)
        results = self.db_cursor.fetchall()

        self.table_widget.setRowCount(len(results))  # 행 개수 설정
        for row_idx, row_data in enumerate(results):
            for col_idx, value in enumerate(row_data):
                self.table_widget.setItem(row_idx, col_idx, QTableWidgetItem(str(value)))

    def closeEvent(self, event):
        """
        창 닫힐 때 데이터베이스 연결 닫기
        """
        self.db_connection.close()
        event.accept()

class KitchenGUI(QMainWindow):
    """
    PyQt5 GUI: 주방 관리 UI
    """
    def __init__(self, ros_node, request_queue):
        super().__init__()
        self.ros_node = ros_node  # ROS2 노드 인스턴스
        self.request_queue = request_queue  # 요청 큐

        # DPI 스케일링 활성화
        QApplication.setAttribute(Qt.AA_EnableHighDpiScaling)
        QApplication.setAttribute(Qt.AA_UseHighDpiPixmaps)

        self.setWindowTitle("Kitchen GUI")  # 윈도우 제목 설정
        self.setGeometry(100, 100, 2560, 1280)  # 윈도우 크기 설정

        # SQLite3 데이터베이스 경로 설정
        self.db_path = "./src/restaurant.db"
        # ros_node.get_logger().info(f"Using database path: {db_path}")  # ROS2 노드 로거로 경로 출력
        
        # 데이터베이스 연결
        self.db_connection = sqlite3.connect(self.db_path) # 데이터베이스 연결
        self.db_cursor = self.db_connection.cursor()

        # 로봇 경로와 테이블 데이터 초기화
        self.robot_path = []  # 로봇 경로

        self.robot_path_num = [] # 전달할 로봇 경로
        self.table_coordinates = {
            1: (1.0, 2.0, 0.0),  # 테이블 1
            2: (3.0, 1.5, 0.0),  # 테이블 2
            3: (4.0, 3.0, 0.0),  # 테이블 3
            4: (0.0, 5.0, 0.0),  # 테이블 4
            5: (2.0, 2.0, 0.0),  # 테이블 5
            6: (3.5, 0.5, 0.0),  # 테이블 6
        }

        self.tables_data = {f"Table{i}": {} for i in range(1, 7)}  # 테이블별 데이터 저장

        # 메인 레이아웃 설정
        main_layout = QHBoxLayout()

        # 좌측 레이아웃: 테이블 및 로봇 경로 관리
        left_layout = QVBoxLayout()

        # 로봇 경로 텍스트박스
        self.robot_path_display = QTextEdit()
        self.robot_path_display.setReadOnly(True)  # 읽기 전용
        left_layout.addWidget(QLabel("로봇 경로"))
        left_layout.addWidget(self.robot_path_display, stretch=1)

        # 테이블 버튼 (6개)
        table_layout = QGridLayout()
        self.table_buttons = {}
        self.clear_buttons = {}  # 결제 버튼 저장

        for i in range(6):
            table_button = QPushButton(f"Table{i+1}\n\n")  # 테이블 정보 버튼
            table_button.setFixedSize(300, 300)  # 버튼 크기 설정
            table_button.clicked.connect(lambda _, t=f"Table{i+1}": self.toggle_table(t, i+1))  # 클릭 이벤트
            self.table_buttons[f"Table{i+1}"] = table_button  # 버튼 저장
            table_layout.addWidget(table_button, i // 2, i % 2)  # 2열 그리드 배치
        
             # 결제 버튼
            clear_button = QPushButton(f"결제 (Table{i+1})")  # 결제 버튼 생성
            clear_button.setFixedSize(300, 50)  # 결제 버튼 크기 설정
            clear_button.clicked.connect(lambda _, t=f"Table{i+1}": self.clear_table_data(t))  # 클릭 이벤트
            self.clear_buttons[f"Table{i+1}"] = clear_button  # 결제 버튼 저장
            table_layout.addWidget(clear_button, i // 2 + 3, i % 2)  # 결제 버튼 추가
        left_layout.addLayout(table_layout, stretch=9)

        # 좌측 레이아웃을 메인 레이아웃에 추가 (50% 비율)
        main_layout.addLayout(left_layout, 5)

        # 중앙 레이아웃: 요청 및 메뉴 접수
        center_layout = QVBoxLayout()

        # 요청 리스트
        self.request_list = QListWidget()
        self.request_list.itemClicked.connect(self.handle_request_completion)  # 요청 완료 처리
        center_layout.addWidget(QLabel("요청 사항"))
        center_layout.addWidget(self.request_list, stretch=1)

        # 메뉴 접수 리스트
        self.menu_list = QListWidget()
        self.menu_list.itemClicked.connect(self.handle_menu_selection)  # 메뉴 선택 처리
        center_layout.addWidget(QLabel("메뉴 접수"))
        center_layout.addWidget(self.menu_list, stretch=2)

        # 중앙 레이아웃을 메인 레이아웃에 추가 (40% 비율)
        main_layout.addLayout(center_layout, 4)

        # 우측 레이아웃: 통계 및 로봇 제어
        right_layout = QVBoxLayout()
        right_layout.setAlignment(Qt.AlignCenter)
        # 버튼 간 간격 추가
        button_spacing = 20

        # 통계 버튼
        stats_button = QPushButton("통계")
        stats_button.setFixedSize(120, 40)  # 크기 설정
        stats_button.clicked.connect(self.open_stats_window)  # 통계 창 연결
        right_layout.addWidget(stats_button)
        right_layout.addSpacing(button_spacing)

        # 편집 버튼
        edit_button = QPushButton("편집")
        edit_button.setFixedSize(120, 40)
        edit_button.clicked.connect(self.edit_table_menu)
        right_layout.addWidget(edit_button)
        right_layout.addSpacing(button_spacing)

        # 호출 버튼
        self.robot_call_button = QPushButton("로봇 호출")
        self.robot_call_button.clicked.connect(self.call_robot)
        self.robot_call_button.setFixedSize(120, 40)
        right_layout.addWidget(self.robot_call_button)
        right_layout.addSpacing(button_spacing)

        # 로봇 출발 버튼
        self.robot_start_button = QPushButton("로봇 출발")
        self.robot_start_button.clicked.connect(self.send_path_robot)
        self.robot_start_button.setFixedSize(120, 40)
        right_layout.addWidget(self.robot_start_button)

        # 우측 레이아웃을 메인 레이아웃에 추가 (10% 비율)
        main_layout.addLayout(right_layout, 1)

        # 메인 레이아웃을 중앙 위젯에 적용
        central_widget = QWidget()
        central_widget.setLayout(main_layout)
        self.setCentralWidget(central_widget)

        # QTimer를 사용해 요청 확인
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.check_requests)  # 요청 확인
        self.timer.start(100)  # 100ms마다 실행

    def call_robot(self):
        self.ros_node.moving_path.append((0, 0, 0))  # KitchenNode의 moving_path에 전달
        self.ros_node.send_next_goal()

    def send_path_robot(self):
        self.moving_path.extend(self.robot_path_num)
        self.robot_path_num=[]
        self.robot_path=[]
        self.update_robot_path_display()

    def send_path_robot(self):
        if not self.robot_path_num:
            self.ros_node.get_logger().warn("No waypoints to send.")
            return

        # 로봇 경로를 moving_path로 전달
        for num in self.robot_path_num:
            x,y,theta = self.table_coordinates[num]
            self.ros_node.moving_path.append((x, y, theta))  # KitchenNode의 moving_path에 전달

        self.robot_path_num = []  # 초기화
        self.robot_path = []
        self.update_robot_path_display()

        # 첫 번째 목표 전송
        self.ros_node.send_next_goal()

    def open_stats_window(self):
        """
        통계 창 열기
        """
        stats_window = StatsWindow(self.db_path, self)
        stats_window.exec_()

    def save_order_to_db(self, menu_name, quantity):
        """
        SQLite3 데이터베이스에 주문 저장
        """
        global menu

        now = datetime.now()  # 현재 날짜와 시간 가져오기
        order_date = now.strftime("%Y-%m-%d")  # 날짜 포맷
        order_time = now.strftime("%H:%M:%S")  # 시간 포맷
        total_price = quantity * menu[menu_name]["price"]

        # 데이터베이스에 삽입
        self.db_cursor.execute("""
            INSERT INTO orders (menu_name, quantity, order_date, order_time, total_price)
            VALUES (?, ?, ?, ?, ?);
        """, (menu_name, quantity, order_date, order_time, total_price))
        self.db_connection.commit()  # 변경 사항 저장

    def toggle_table(self, table_name, table_num):
        """테이블 클릭 시 로봇 경로 추가/삭제"""
        if table_name in self.robot_path:
            self.robot_path.remove(table_name)  # 경로에서 제거
            self.robot_path_num.remove(table_num)
        else:
            self.robot_path.append(table_name)  # 경로에 추가
            self.robot_path_num.append(table_num)
        self.update_robot_path_display()


    def update_robot_path_display(self):
        """로봇 경로 실시간 업데이트"""
        self.robot_path_display.setText(" -> ".join(self.robot_path))  # 경로 표시

    def handle_request_completion(self, item):
        """요청 사항 완료 처리"""
        if item:
            reply = QMessageBox.question(
                self, "요청 완료", f"{item.text()}를 완료하시겠습니까?",
                QMessageBox.Yes | QMessageBox.No
            )
            if reply == QMessageBox.Yes:
                self.request_list.takeItem(self.request_list.row(item))  # 요청 리스트에서 제거

    def handle_menu_selection(self, item):
        """메뉴 접수/거절 처리"""
        if item:
            dialog = QDialog(self)
            dialog.setWindowTitle("메뉴 처리")
            layout = QVBoxLayout(dialog)

            label = QLabel(f"{item.text()}를 어떻게 처리하시겠습니까?")
            layout.addWidget(label)

            accept_button = QPushButton("접수")
            accept_button.clicked.connect(lambda: self.process_menu(item, True, dialog))
            
            reject_button = QPushButton("거절")
            reject_button.clicked.connect(lambda: self.process_menu(item, False, dialog))

            layout.addWidget(accept_button)
            layout.addWidget(reject_button)
            dialog.setLayout(layout)
            dialog.exec_()

    def process_menu(self, item, action, dialog):
        """
        메뉴를 접수 또는 거절
        """
        if action:
            table_name, menu_data = item.text().split(": ", 1)
            menu_name, quantity_str = menu_data.split(" x ")
            quantity = int(quantity_str)  # 수량 변환
            
            # 기존 메뉴가 있는지 확인하고 수량 합산
            if menu_name in self.tables_data[table_name]:
                self.tables_data[table_name][menu_name] += quantity
            else:
                self.tables_data[table_name][menu_name] = quantity
            
            # 테이블 UI 업데이트
            self.update_table_display(table_name)

            # SQLite3에 주문 저장
            self.save_order_to_db(menu_name, quantity)

        self.menu_list.takeItem(self.menu_list.row(item))
        dialog.accept()

    def closeEvent(self, event):
        """
        GUI 종료 시 데이터베이스 연결 닫기
        """
        self.db_connection.close()
        event.accept()

    def update_table_display(self, table_name):
        """
        테이블 메뉴 정보 업데이트
        """
        menu_info = "\n".join(
            f"{menu_name} - 수량: {quantity}"
            for menu_name, quantity in self.tables_data[table_name].items()
        )
        self.table_buttons[table_name].setText(f"{table_name}\n\n{menu_info}")

    def check_requests(self):
        """Queue에서 요청 확인"""
        while not self.request_queue.empty():
            request = self.request_queue.get()
            if "Table 요청" in request:
                self.request_list.addItem(request)  # 요청 리스트에 추가
            else:
                self.menu_list.addItem(request)  # 메뉴 리스트에 추가

    def clear_table_data(self, table_name):
        """
        특정 테이블 데이터를 초기화하고 결제 금액을 보여주는 팝업창을 표시
        """
        # 테이블에 있는 메뉴들의 총 금액 계산
        total_amount = 0
        for menu_name, quantity in self.tables_data[table_name].items():
            total_amount += menu[menu_name]["price"] * quantity  # 가격 * 수량

        # 결제 금액 팝업창 표시
        reply = QMessageBox.question(
            self, "결제", f"{table_name}의 결제 금액은 {total_amount}원입니다. 결제를 완료하시겠습니까?",
            QMessageBox.Yes | QMessageBox.No
        )

        # 결제 완료 시 데이터 초기화
        if reply == QMessageBox.Yes:
            self.tables_data[table_name] = {}  # 데이터 초기화
            self.update_table_display(table_name)  # UI 업데이트
            QMessageBox.information(self, "완료", f"{table_name}의 결제가 완료되었습니다.")

    def edit_table_menu(self):
        """
        테이블 메뉴 편집: 이동할 소스 테이블과 대상 테이블을 선택
        """
        dialog = QDialog(self)
        dialog.setWindowTitle("테이블 메뉴 편집")
        dialog_layout = QVBoxLayout(dialog)

        label = QLabel("먼저 이동할 메뉴가 있는 테이블을 선택하세요.")
        dialog_layout.addWidget(label)

        # 소스 테이블 버튼 생성
        for table_name in self.tables_data.keys():
            button = QPushButton(f"{table_name}")
            button.clicked.connect(lambda _, t=table_name, d=dialog: self.select_source_table(t, d))
            dialog_layout.addWidget(button)

        dialog.setLayout(dialog_layout)
        dialog.exec_()

    def select_source_table(self, source_table, dialog):
        """
        소스 테이블 선택 후 대상 테이블 선택 다이얼로그 열기
        """
        if not self.tables_data[source_table]:  # 소스 테이블에 메뉴가 없을 경우
            QMessageBox.warning(self, "오류", f"{source_table}에는 메뉴가 없습니다.")
            return

        # 대상 테이블 선택 다이얼로그 생성
        target_dialog = QDialog(self)
        target_dialog.setWindowTitle("대상 테이블 선택")
        target_dialog_layout = QVBoxLayout(target_dialog)

        label = QLabel(f"{source_table}에서 메뉴를 이동할 대상 테이블을 선택하세요.")
        target_dialog_layout.addWidget(label)

        # 대상 테이블 버튼 생성
        for table_name in self.tables_data.keys():
            if table_name != source_table:  # 소스 테이블과 다른 테이블만 표시
                button = QPushButton(f"{table_name}")
                button.clicked.connect(lambda _, s=source_table, t=table_name, d=target_dialog: self.move_menu_to_table(s, t, d))
                target_dialog_layout.addWidget(button)

        target_dialog.setLayout(target_dialog_layout)
        target_dialog.exec_()
    
    def move_menu_to_table(self, source_table, target_table, dialog):
        """
        소스 테이블에서 대상 테이블로 메뉴 이동
        """
        # 메뉴 데이터를 합산하여 이동
        for menu_name, quantity in self.tables_data[source_table].items():
            if menu_name in self.tables_data[target_table]:
                self.tables_data[target_table][menu_name] += quantity
            else:
                self.tables_data[target_table][menu_name] = quantity

        # 소스 테이블 데이터 초기화
        self.tables_data[source_table] = {}

        # UI 업데이트
        self.update_table_display(source_table)
        self.update_table_display(target_table)

        QMessageBox.information(self, "완료", f"{source_table}의 메뉴가 {target_table}로 이동되었습니다.")
        dialog.accept()


def main():
    """
    메인 실행 함수
    """
    rclpy.init()  # ROS2 초기화
    request_queue = queue.Queue()  # 요청 큐 생성
    ros_node = KitchenNode(request_queue)  # ROS 노드 생성

    # ROS2 스레드 관리
    running = threading.Event()
    running.set()

    def ros_spin():
        while running.is_set():
            rclpy.spin_once(ros_node, timeout_sec=0.1)  # ROS 메시지 처리

    ros_thread = threading.Thread(target=ros_spin, daemon=True)  # ROS 스레드 생성
    ros_thread.start()

    # PyQt5 실행
    app = QApplication(sys.argv)
    gui = KitchenGUI(ros_node, request_queue)
    gui.show()

    try:
        sys.exit(app.exec_())  # PyQt 애플리케이션 실행
    except KeyboardInterrupt:
        print("종료 중...")
    finally:
        running.clear()  # 스레드 종료
        ros_thread.join()
        ros_node.destroy_node()  # 노드 종료
        rclpy.shutdown()  # ROS2 종료

if __name__ == "__main__":
    main()
